In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import folium
import openrouteservice as ors
from geopy.distance import great_circle

In [2]:
def get_token():
    load_dotenv('./.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    client_id = os.environ.get("X-ClientId")
    pass_key = os.environ.get("passKey")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [3]:
get_token()

b'{"code": "00", "description": "Register user: silviluliuma with token: 70d9c557-201e-41c1-a674-f0e8953d0d22  Data recovered  OK, (lapsed: 371 millsecs)", "datetime": "2024-01-31T12:53:01.005436", "data": [{"updatedAt": "2024-01-09T17:02:16.9230000", "username": "silviluliuma", "accessToken": "70d9c557-201e-41c1-a674-f0e8953d0d22", "email": "valeromsilvia@gmail.com", "idUser": "26ddfea8-f2ce-438f-89e4-5110c9abd07e", "tokenSecExpiration": 86399, "apiCounter": {"current": 0, "dailyUse": 20000, "owner": 0, "licenceUse": "Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!", "aboutUses": "If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylabs.emtmadrid.es/doc/new-app and https://apidocs.emtmadrid.es/#api-Block_1_User_identity-login)"}, "nameApp": "OPENAPI MobilityLabs", "priv": "U"}]}'

In [6]:
def get_stations():
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    print(token)
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [7]:
get_stations()

8110f13b-8027-42d9-82f8-7b1ad7b05179


{'code': '80',
 'description': 'Error, token not found in cache',
 'datetime': '2024-01-31T12:53:37.383614',
 'data': []}

In [8]:
stations = get_stations()
stations_real_time = pd.DataFrame(stations["data"])
stations_real_time


8110f13b-8027-42d9-82f8-7b1ad7b05179


,activate,address,dock_bikes,free_bases,geofence,geometry,id,integrator,light,name,...,reservations_count,tipo_estacionPBSC,total_bases,virtualDelete,virtual_bikes,virtual_bikes_num,code_district,code_suburb,geofenced_capacity,bikesGo
0,1,"Avenida del Ensanche de Vallecas, 9,",18,9,"{'type': 'Polygon', 'coordinates': [[[-3.61311...","{'type': 'Point', 'coordinates': [-3.61253028,...",2190,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,2,"453 - Avenida del Ensanche de Vallecas, 9",...,0,FIXED,27,False,[],0,18,183,20,[]
1,1,"Paseo de la Chopera,33,Comunidad de Madrid España",13,10,"{'type': 'Polygon', 'coordinates': [[[-3.70098...","{'type': 'Point', 'coordinates': [-3.700399664...",2205,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,2,"267 - Paseo de la Chopera, 33",...,0,FIXED,23,False,[],0,02,023,20,[]
2,1,"Paseo de la Castellana nº 164,Comunidad de Mad...",21,2,"{'type': 'Polygon', 'coordinates': [[[-3.69000...","{'type': 'Point', 'coordinates': [-3.6894151, ...",2225,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,1,157 - Castellana 164.,...,0,FIXED,23,False,[],0,05,055,20,[]
3,1,"Guetaria , 84b,",15,9,"{'type': 'Polygon', 'coordinates': [[[-3.71628...","{'type': 'Point', 'coordinates': [-3.71569102,...",2226,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,2,395b - 395 - Guetaria 84b,...,0,FIXED,24,False,[],0,12,121,20,[]
4,1,"Calle de Gloria Fuertes, 181,",20,7,"{'type': 'Polygon', 'coordinates': [[[-3.72554...","{'type': 'Point', 'coordinates': [-3.7249527, ...",2151,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,1,"592 - Calle de Gloria Fuertes, 181",...,0,FIXED,27,False,[],0,08,087,20,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,1,"Calle María de las Mercedes de Borbón, 98,",11,10,"{'type': 'Polygon', 'coordinates': [[[-3.61269...","{'type': 'Point', 'coordinates': [-3.612106853...",2366,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,2,"611 - María de las Mercedes de Borbón, 98",...,0,FIXED,23,False,[],0,16,166,20,[]
608,1,"Calle Charleroi,",1,23,"{'type': 'Polygon', 'coordinates': [[[-3.63278...","{'type': 'Point', 'coordinates': [-3.63219372,...",2075,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,0,434 - Calle Charleroi,...,0,FIXED,24,False,[],0,13,134,20,[]
609,1,"340 - Calle Illescas, 1C,",7,20,"{'type': 'Polygon', 'coordinates': [[[-3.76628...","{'type': 'Point', 'coordinates': [-3.7656976, ...",2334,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,0,"340 - Calle Illescas, 1C",...,0,FIXED,27,False,[],0,10,104,0,[]
610,1,"Calle de Bohemios, 1,",19,5,"{'type': 'Polygon', 'coordinates': [[[-3.69449...","{'type': 'Point', 'coordinates': [-3.693901, 4...",2062,3c34ea72-2a03-4a3c-be97-cf32be4b0ebe,1,"399 - Calle de Bohemios, 1",...,0,FIXED,24,False,[],0,17,175,20,[]


In [ ]:
stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
stations_real_time = stations_real_time.drop(["geofence", "activate", "geometry", "integrator", "reservations_count", "no_available", "tipo_estacionPBSC", "virtualDelete", "virtual_bikes", "virtual_bikes_num", "code_suburb", "geofenced_capacity", "bikesGo"], axis=1)
stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
stations_real_time

In [7]:
def get_district(df, district_number):
    result = df[df["code_district"] == district_number]
    return result

district_03 = get_district(stations_real_time, "03")
district_03

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
62,"Calle O'Donnell nº 28,",10,17,1465,2,61 - Narváez,61,27,03,-3.675314,40.421455,"(-3.6753139, 40.4214551)"
63,"Calle Ibiza nº 62,",5,18,1467,0,63 - Ibiza,63,23,03,-3.670896,40.417924,"(-3.6708959, 40.4179237)"
64,"Calle Antonio Maura nº 15,",11,13,1468,2,65 - Antonio Maura,65,24,03,-3.689425,40.416701,"(-3.689424797942769, 40.4167014469994)"
65,"Calle Espalter nº 1,",2,17,1470,0,68 - Espalter,68,19,03,-3.691323,40.412769,"(-3.6913232, 40.4127692)"
66,"Avenida de Alfonso XII nº 54,",11,16,1471,2,69 - Puerta del Ángel Caído,69,27,03,-3.688822,40.409808,"(-3.688822, 40.409808)"
67,"Avenida de Menéndez Pelayo nº 63,",26,1,1472,1,70 - Puerta del Doce de Octubre,70,27,03,-3.677873,40.415382,"(-3.677872542327878, 40.415382385574716)"
68,"Avenida de Nazaret nº 7,",6,17,1475,0,73 - Plaza de los Astros,73,23,03,-3.668700,40.411500,"(-3.6687, 40.4115)"
69,"Avenida de Menéndez Pelayo nº 69,",14,13,1476,2,74 - Puerta del Pacífico,74,27,03,-3.676589,40.412063,"(-3.676588763788984, 40.41206290978386)"
70,Avenida de Menéndez Pelayo con calle Poeta Est...,25,2,1477,1,75 - Menéndez Pelayo,75,27,03,-3.678475,40.408272,"(-3.6784749, 40.4082717)"
71,"Calle Doctor Esquerdo nº 191,",22,1,1487,1,84 - Valderribas,84,23,03,-3.672482,40.403572,"(-3.672481575393678, 40.40357156992851)"


In [8]:
district_01 = stations_real_time[stations_real_time["code_district"]== "01"]
district_02 = stations_real_time[stations_real_time["code_district"]== "02"]
district_03 = stations_real_time[stations_real_time["code_district"]== "03"]
district_04 = stations_real_time[stations_real_time["code_district"]== "04"]
district_05 = stations_real_time[stations_real_time["code_district"]== "05"]
district_06 = stations_real_time[stations_real_time["code_district"]== "06"]
district_07 = stations_real_time[stations_real_time["code_district"]== "07"]
district_08 = stations_real_time[stations_real_time["code_district"]== "08"]
district_09 = stations_real_time[stations_real_time["code_district"]== "09"]
district_10 = stations_real_time[stations_real_time["code_district"]== "10"]
district_11 = stations_real_time[stations_real_time["code_district"]== "11"]
district_12 = stations_real_time[stations_real_time["code_district"]== "12"]
district_13 = stations_real_time[stations_real_time["code_district"]== "13"]
district_14 = stations_real_time[stations_real_time["code_district"]== "14"]
district_15 = stations_real_time[stations_real_time["code_district"]== "15"]
district_16 = stations_real_time[stations_real_time["code_district"]== "16"]
district_17 = stations_real_time[stations_real_time["code_district"]== "17"]
district_18 = stations_real_time[stations_real_time["code_district"]== "18"]
district_19 = stations_real_time[stations_real_time["code_district"]== "19"]
district_20 = stations_real_time[stations_real_time["code_district"]== "20"]
district_21 = stations_real_time[stations_real_time["code_district"]== "21"]

In [9]:
district_02

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",19,4,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
57,"Calle Ribera de Curtidores nº 28,",25,2,1450,1,46 - Ribera de Curtidores,46,27,02,-3.707126,40.405315,"(-3.7071259, 40.4053153)"
99,Calle Teresa López Valcárcel esq. Teniente Cor...,19,5,2071,1,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
136,"Calle Delicias, 45,",7,12,2117,2,"273 - Calle Delicias, 45",273,19,02,-3.690918,40.403819,"(-3.69091831, 40.40381914)"
181,"Glorieta de Embajadores nº 6,",20,4,1451,1,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
203,"PASEO DE LOS MELANCÓLICOS 73,",7,16,2247,0,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,02,-3.718569,40.401782,"(-3.718569, 40.401782)"
216,"271 - Calle Amaltea , 1,",18,6,2265,1,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706,"(-3.68317, 40.394706)"
218,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",9,14,2267,2,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,23,02,-3.708212,40.402054,"(-3.7082123, 40.4020545)"
220,"Calle Méndez Alvaro, 44,",5,18,2269,0,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"
279,"Paseo de los Pontones, 45,",6,17,2299,0,"278 - Paseo de los Pontones, 45",278,23,02,-3.719600,40.404043,"(-3.7196, 40.4040433)"


In [10]:
def get_light1(df):
    df_light1 = df[df["light"]==1]
    return df_light1

In [11]:
get_light1(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",19,4,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
57,"Calle Ribera de Curtidores nº 28,",25,2,1450,1,46 - Ribera de Curtidores,46,27,02,-3.707126,40.405315,"(-3.7071259, 40.4053153)"
99,Calle Teresa López Valcárcel esq. Teniente Cor...,19,5,2071,1,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
181,"Glorieta de Embajadores nº 6,",20,4,1451,1,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
216,"271 - Calle Amaltea , 1,",18,6,2265,1,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706,"(-3.68317, 40.394706)"
285,"Paseo virgen de puerto, 25,",16,0,2304,1,"277 - Paseo Virgen del Puerto, 25",277,15,02,-3.721178,40.411622,"(-3.721178, 40.411622)"
305,"279 - Paseo de los Melancólicos, 1,",22,1,2331,1,"279 - Paseo de los Melancólicos, 1",279,23,02,-3.718482,40.410445,"(-3.7184816, 40.4104448)"
324,"Calle del tejo,",20,3,2305,1,272 - Calle del tejo,272,23,02,-3.679186,40.396630,"(-3.679186, 40.39663)"
351,"Calle Palos de la Frontera nº 27,",23,4,1526,1,120 - Palos de la Frontera,120,27,02,-3.694825,40.403090,"(-3.6948251018524214, 40.403090451430614)"
352,"Paseo de Santa María de la Cabeza nº 58,",24,3,1527,1,121 - Santa María de la Cabeza,121,27,02,-3.698603,40.401749,"(-3.6986027, 40.4017487)"


In [12]:
def get_light0(df):
    df_light0 = df[df["light"]==0]
    return df_light0

In [13]:
get_light0(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
203,"PASEO DE LOS MELANCÓLICOS 73,",7,16,2247,0,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,02,-3.718569,40.401782,"(-3.718569, 40.401782)"
220,"Calle Méndez Alvaro, 44,",5,18,2269,0,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"
279,"Paseo de los Pontones, 45,",6,17,2299,0,"278 - Paseo de los Pontones, 45",278,23,02,-3.719600,40.404043,"(-3.7196, 40.4040433)"
337,"Calle del Cobre (Metro Arganzuela Planetario),",7,16,2017,0,268 - Calle del Cobre (M. Planetario),268,23,02,-3.689031,40.393383,"(-3.689031463575172, 40.39338327980599)"
464,"Calle Retama nº 5,",5,18,1586,0,180 - Méndez Álvaro,180,23,02,-3.675890,40.394042,"(-3.67589, 40.3940423)"


In [14]:
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, vehicle_start).meters)
    return nearest_station

In [15]:
load_dotenv('../.env')
client = ors.Client(key=os.environ.get("openroute_api_key"))

# Function to find the nearest station to given coordinates
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, coords).meters)
    return nearest_station

# Function to create a route between two sets of coordinates
def create_route(client, start_coords, end_coords):
    return client.directions(
        coordinates=[start_coords, end_coords],
        profile='driving-car',
        format='geojson'
    )

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

# Creo un mapa con Folium
m = folium.Map(location=[vehicle_start[1], vehicle_start[0]], zoom_start=12)

# Añado marcador morado para la central (vehicle_start)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='CENTRAL EMT', icon=folium.Icon(color='purple')).add_to(m)

distrito02_low = get_light0(district_02).copy()
distrito02_high = get_light1(district_02).copy()

distrito02_high['visited'] = False
distrito02_low['visited'] = False

# Inicio de la ruta
current_coords = vehicle_start
van = "empty"

coords_list = [current_coords]

stop_counter = 1

for i in range(100):
    if van == "empty":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)].empty:
            nearest_station = find_nearest_to_coords(distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'visited'] = True
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "full"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with high occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='orange')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
    elif van == "full":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)].empty:
            nearest_station = find_nearest_to_coords(distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'visited'] = True
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "empty"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with low occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='green')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
final_route = create_route(client, coords_list[-1], vehicle_start)
folium.PolyLine(locations=[coord[::-1] for coord in final_route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)

m